Cleaning code from https://ryanwingate.com/projects/machine-learning-data-prep/adult/adult-cleaning/.

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [115]:
adult_data_path = 'raw_adult/adult.data'
adult_test_path = 'raw_adult/adult.test'
cols = ['age','workclass','fnlwgt','education','education-num','marital-status',
        'occupation','relationship','race','sex','capital-gain', 'capital-loss',
        'hours-per-week', 'native-country','compensation']
a = (pd.read_csv(adult_data_path,
                 names=cols,
                 sep=', ',
                 engine='python')
     .append(pd.read_csv(adult_test_path,
                         skiprows=1,
                         names=cols,
                         sep=', ',
                         engine='python')))
print(a.shape)
a.head()

(48842, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,compensation
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [116]:
a = a.replace({'<=50K.' : '<=50K',
               '>50K.'  : '>50K'})

# Drop entries where workclass and occupation are unknown, and where workclass is Without-pay.
a = (a[(a['workclass']!='?')&
       (a['occupation']!='?')&
       (a['workclass']!='Without-pay')]
     .reset_index(drop=True))
a['idx'] = a.index

# Map the very small Armed-Forces category of occupation to Protective-serv.
a.loc[a['occupation']=='Armed-Forces','occupation'] = 'Protective-serv'

# Map Ages, Education, Workclass, and Weekly-Hours to smaller category set.
a.loc[a['workclass'].isin(['State-gov', 'Federal-gov', 'Local-gov']), 
      'employment-type'] = 'Government'
a.loc[a['workclass'].isin(['Self-emp-not-inc', 'Self-emp-inc']),      
      'employment-type'] = 'Self-Employed'
a.loc[a['workclass'].isin(['Private']),                               
      'employment-type'] = 'Privately-Employed'

# a.loc[a['education-num'] <= 8,          'education'] = 'Less than High School'
# a.loc[a['education-num'].isin([ 9,10]), 'education'] = 'High School'
# a.loc[a['education-num'].isin([11,12]), 'education'] = 'Associates'
# a.loc[a['education-num'].isin([13]),    'education'] = 'Bachelors'
# a.loc[a['education-num'].isin([14]),    'education'] = 'Masters'
# a.loc[a['education-num'].isin([15,16]), 'education'] = 'PhD/Professional'

a = a.rename(columns={'hours-per-week':'weekly-hours'})
a.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,weekly-hours,native-country,compensation,idx,employment-type
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0,Government
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1,Self-Employed
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,2,Privately-Employed
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,3,Privately-Employed
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,4,Privately-Employed


In [117]:
# Bin age and weekly hours using percentiles

for var in ['age', 'weekly-hours']:
    a[f'{var} raw'] = a[var]
    bin_edges = np.unique(np.percentile(a[f'{var} raw'].values, [0,5,15,25,25,45,55,65,75,85,95,97.5,100]))
    print(var, bin_edges)

    a[f'{var}-bin'] = pd.cut(a[f'{var} raw'], bins=bin_edges, include_lowest=True, labels=False)

    # plt.figure()
    # plt.hist(a[f'{var} raw'].values)
    
a[["age-bin","age raw", "weekly-hours-bin", "weekly-hours raw"]].head(200)

age [17. 20. 24. 28. 35. 39. 43. 47. 53. 62. 67. 90.]
weekly-hours [ 1. 20. 32. 40. 45. 50. 60. 70. 99.]


,age-bin,age raw,weekly-hours-bin,weekly-hours raw
0,4,39,2,40
1,7,50,0,13
2,4,38,2,40
3,7,53,2,40
4,2,28,2,40
...,...,...,...,...
195,4,36,2,36
196,3,35,5,60
197,8,58,2,35
198,0,17,0,12


In [118]:
# Convert remaining categorical columns to numerical labels

VARS = ['sex', 'compensation', 'education-num'] # Always do these
ONE_HOT = False
if not ONE_HOT: VARS += ['employment-type', 'occupation', 'race'] # Do these if *not* doing one-hot encoding

for var in VARS:
       a[f'{var} raw'] = a[var]
       a[var] = a[var].astype('category')
       a[var] = a[var].cat.codes
       print(a[[var,f'{var} raw','idx']].groupby([var, f'{var} raw']).count())

a = a[['idx',
       # 'age',
       'age-bin',
       'employment-type',
       # 'education',
       'education-num',
       'occupation',
       'race',
       'sex',
       # 'weekly-hours',
       'weekly-hours-bin',
       'compensation']].copy()
print(a.shape)

               idx
sex sex raw       
0   Female   14912
1   Male     31100
                                 idx
compensation compensation raw       
0            <=50K             34592
1            >50K              11420
                                   idx
education-num education-num raw       
0             1                     73
1             2                    229
2             3                    468
3             4                    842
4             5                    687
5             6                   1239
6             7                   1631
7             8                    599
8             9                  14958
9             10                 10033
10            11                  1978
11            12                  1527
12            13                  7772
13            14                  2590
14            15                   810
15            16                   576
                                       idx
employment-type employment-type

In [119]:
a.head()

,idx,age-bin,employment-type,education-num,occupation,race,sex,weekly-hours-bin,compensation
0,0,4,0,12,0,4,1,2,0
1,1,7,2,12,2,4,1,0,0
2,2,4,1,8,4,4,1,2,0
3,3,7,1,6,4,2,1,2,0
4,4,2,1,12,8,2,0,2,0


In [46]:
if ONE_HOT:
    # One-hot encoding of categorical variables.
    print(a.shape)
    a = pd.get_dummies(a, columns=['employment-type', 'occupation', 'race'], drop_first=False)
    print(a.shape)
a.head()

,idx,age,employment-type,education-num,occupation,race,sex,weekly-hours,compensation
0,0,39,0,13,0,4,1,40,0
1,1,50,2,13,2,4,1,13,0
2,2,38,1,9,4,4,1,40,0
3,3,53,1,7,4,2,1,40,0
4,4,28,1,13,8,2,0,40,0


In [120]:
# Move compensation to the end.
_c = a.pop('compensation') 
a['compensation'] = _c

a.to_csv(f"adult{'_one_hot' if ONE_HOT else ''}.csv", index=False)

---
### Don't do these ones.

In [5]:
# Normalize Continuous Data.
print('   Mean Age = ' + str(a['age'].mean()))
print('Std Dev Age = ' + str(a['age'].std()))
a['age'] = (a['age'] - a['age'].mean()) / a['age'].std()

print('   Mean Weekly-Hours = ' + str(a['weekly-hours'].mean()))
print('Std Dev Weekly-Hours = ' + str(a['weekly-hours'].std()))
a['weekly-hours'] = (a['weekly-hours'] - a['weekly-hours'].mean()) / a['weekly-hours'].std()
a.head()


   Mean Age = 38.55261670868469
Std Dev Age = 13.199319129893762
   Mean Weekly-Hours = 40.95159958271755
Std Dev Weekly-Hours = 12.00779742417576


,idx,age,employment-type,education,occupation,race,sex,weekly-hours,compensation
0,0,0.033894,0,1,0,4,Male,-0.079248,0
1,1,0.867271,2,1,2,4,Male,-2.327787,0
2,2,-0.041867,1,2,4,4,Male,-0.079248,0
3,3,1.094555,1,3,4,2,Male,-0.079248,0
4,4,-0.799482,1,1,8,2,Female,-0.079248,0
